This notebook was used for phase 2 to create difficult cases for the submission requirement

# Phase 2: Break It

In [ ]:
import csv
import pandas as pd
import re

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
#!python3 -m spacy download en_core_web_sm
import spacy
import json

In [ ]:
TRAIN = '../data/interim/train.csv'

In [ ]:
data_train = pd.read_json('../data/raw/music_reviews_train.json.gz', lines=True)
data_train['sentiment']

In [ ]:
labels = data_train['sentiment'].tolist()

In [ ]:
input_data = []
label = [] 
c = 0
n = 100 # set how many training examples to use.
with open(TRAIN, mode='r') as file:
    csvFile = csv.reader(file)
    for lines in csvFile:
        if c < n:
            input_data.append(lines[0])
            label.append(int(lines[1]))
            c+=1
        else:
            break

In [ ]:
def tkn(sentence):
    """Function to find all tokens in a given sentence
    """
    tok = re.compile('[\'\"]|[A-Za-z]+|[.?!:\'\"]+')
    
    return tok.findall(sentence)
    
def splitsies(para):
    punct = re.compile('[.?!:]')
    t = punct.split(para)
    spl= []
    for i in t:
        temp = tkn(i)
        if len(temp) > 0:
            spl.append(temp)
        
    return spl
    
#splitsies(input_data[0])

In [ ]:
print(f'Sentence 1:\n{input_data[0]}\nLabel: {label[0]}')
type(input_data)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

token_text = []
for para in input_data:
    token_text.append(sent_tokenize(para))
token_text[0]

In [ ]:
nlp = spacy.load('en_core_web_sm')

# Add Typos

In [ ]:
ret = Perturb.perturb(input_data, Perturb.add_typos)
ret.data[0]

In [ ]:
# This just typos one sentence in input paragraph
typo = []
for thing in ret.data:
    typo.append(thing[1])
typo[0]
#typo

In [ ]:
# If "sentence" in para is only 1 char, add ".." to avoid errors
for i in range(len(token_text)):
    for j in range(len(token_text[i])):
        if len(token_text[i][j]) <= 1:
            token_text[i][j] = token_text[i][j]+'..'
            
token_text[6]

In [ ]:
# Typo every sentence in every paragraph
typoed = []
lala = []
for i in range(len(token_text)):
    ret = Perturb.perturb(token_text[i], Perturb.add_typos)
    typos = []
    for sent in ret.data:
        typos.append(sent[1])
        lala.append(sent[1])
    typoed.append(typos)
typoed[6]
#typoed

In [ ]:
# Convert back to list of paragraphs WOWWWW
tp = []
for i in range(len(typoed)):
    para = ""
    for j in range(len(typoed[i])):
        para = para + typoed[i][j] + " "
    tp.append(para)
#tp

# POS tag data

https://spacy.io/usage/processing-pipelines

When you call nlp on a text, spaCy will tokenize it and then call each component on the Doc, in order. It then returns the processed Doc that you can work with.

In [ ]:
pdata = list(nlp.pipe(tp))
#for doc in nlp.pipe(input_data):
    # Do something with the doc here
#    print([(ent.text, ent.label_) for ent in doc.ents])

# Remove End Punctuation

In [ ]:
pdata[0], Perturb.strip_punctuation(pdata[0])

In [ ]:
ret = Perturb.perturb(pdata, Perturb.punctuation)
no_punct = []
for i in ret.data:
    no_punct.append(i[1])
no_punct[0]
len(no_punct)
no_punct[0]

# Negation

In [ ]:
no_punct[0]

In [ ]:
pdata = list(nlp.pipe(no_punct))
pdata[0]

In [ ]:
# This negates only 44 sentences
nega = []
count = 0
for i in range(100):
    ret = Perturb.remove_negation(pdata[i])
    if ret == None:  # if nothing changes
        nega.append(pdata[i]) # append original sentence
    else: 
        nega.append(ret) # append negated paragraph
        count +=1
len(nega)

# Negation doesn't really change the sentiment labels, so they remain the same

In [ ]:
count

In [ ]:
nega

# Make Final Output

In [ ]:
output = []
for i in range(len(input_data)):
    dicti = {}
    dicti['reviewText'] = str(nega[i])
    dicti['sentiment'] = labels[i]
    dicti['category'] = "{'typos', 'punct', 'negation'}"
    output.append(dicti)
len(output)
print(output[0])

In [ ]:
#test_json = [json.dumps(i)+'\n' for i in output]
#with open ('../data/predictions/dee_dump.json', 'w') as file:
#    file.writelines(test_json)